In [5]:
import pandas as pd
import collections
import etl_idesp

import numpy as np
from fklearn.training.regression import xgb_regression_learner
from fklearn.training.pipeline import build_pipeline
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [4]:
df_idesp = etl_idesp.etl()
df_idesp.id_escola = df_idesp.id_escola.astype(int)

In [7]:
arquivo = 'D:\\DESAFIO_SEDUC\\dados-VERSAO_1\\escolas_enderecos\\11_escolas_enderecos_0.csv'
with open(arquivo, encoding="utf8") as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
dados_lines = []
len_lines = len(lines[0].split(','))
for line in lines[1:]:
    line = line.rstrip().split(',')
    if len(line) == len_lines:  # linhas com numero incorreto de colunas foram descartadas (1)
        dados_lines.append(line)
end = pd.DataFrame(data=dados_lines)
end.columns = lines[0].split(',')
end.head(3)

,nomedep,depadm,mun,codmun,de,CD_ESCOLA,CD_DIRETORIA,CD_DIRETORIA_ESTADUAL,CD_DIRETORIA_SUPVS_PROPR,NM_COMPLETO_ESCOLA,CD_UNIDADE,DS_ENDERECO,COMPLEMENTO,NUMERO,CD_ORIGEM_UNIDADE,CD_ORIGEM_ESCOLA,CD_ORIGEM_ENDERECO,LATITUDE,LONGITUDE
0,ESTADUAL - SE,1,SAO PAULO,100,NORTE 1,36444,10101,10101,10101,GENESIO DE ALMEIDA MOURA DOUTOR,37806,DOMINGOS AREVALO,RUA,862,37806,36444,37866,-23.447,-46.6967
1,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,1582,10205,10205,10205,DOMINGOS FAUSTINO SARMIENTO,24902,21 DE ABRIL,RUA,970,24902,1582,24962,-23.5422317000605,-46.6057931815944
2,ESTADUAL - SE,1,SAO PAULO,100,LESTE 5,2173,10205,10205,10205,JOAO BORGES PROFESSOR,24901,ITAPURA,RUA,976,24901,2173,24961,-23.5473,-46.566


In [8]:
end = end.rename({'CD_ESCOLA': 'id_escola'}, axis=1)
end = end.rename({'LONGITUDE\n': 'LONGITUDE'}, axis=1)
end.id_escola = end.id_escola.astype(int)
escolas_end = set(end.id_escola)
end = end.drop_duplicates('id_escola')

In [34]:
idesp = pd.merge(df_idesp,end, on='id_escola',how='left')

In [42]:
idesp = idesp.dropna(subset=['nota_ano','LONGITUDE','LATITUDE'])
idesp.LONGITUDE = idesp.LONGITUDE.astype(float)
idesp.LATITUDE = idesp.LATITUDE.astype(float)
idesp.nota_ano = idesp.nota_ano.astype(float)
idesp = idesp.drop_duplicates('id_escola')

In [59]:
dados = idesp.loc[:,['id_gestao', 'nota_ano','LONGITUDE','LATITUDE']].copy()

In [60]:
# SEPARACAO ENTRE TREINO E TESTE
dados_teste = dados.sample(int(len(dados)/6), replace = False)
index_teste = dados_teste.index
dados_treino = dados.drop(index_teste)
dados_teste = dados_teste.reset_index(drop=True)
dados_treino = dados_treino.reset_index(drop=True)
print('dados:  ', len(dados), '  treino:  ', len(dados_treino), '  teste:  ', len(dados_teste))

dados:   5097   treino:   4248   teste:   849


In [61]:
AUXILIARY = ['id_gestao']
TARGET = ['nota_ano']
FEATURES = set(dados.columns) - set(AUXILIARY) - set(TARGET)

my_model = xgb_regression_learner(
    features=FEATURES,
    target=TARGET[0],
    prediction_column='prediction',
    extra_params={'seed': 139, 'nthread': 8},
)
my_learner = build_pipeline(my_model)

# TREINO
(prediction_function, treinado, logs) = my_learner(dados_treino)
# TESTE
testados = prediction_function(dados_teste)

In [62]:
r2_score(testados.nota_ano, testados.prediction.tolist())

0.06509944597552875

In [63]:
r2_score(treinado.nota_ano.tolist(), treinado.prediction.tolist())

0.36597987459181613